### Project 1 : COEUR

In [32]:
import numpy as np
import pandas as pd

# import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, classification_report

import pickle

#### Charge le dataset coeur.

In [2]:
# Affiche le répertoire courant.
# os.getcwd()

# Change de répertoire de travail.
# os.chdir(r"c:/Users/...")

In [3]:
# Charge le dataset.
data = pd.read_excel("dataset/coeur.xlsx")

In [4]:
# Copie data dans df.
df = data.copy()

In [5]:
# Affiche les informations relatives à df.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AGE          918 non-null    int64  
 1   SEXE         918 non-null    object 
 2   TDT          918 non-null    object 
 3   PAR          918 non-null    int64  
 4   CHOLESTEROL  918 non-null    int64  
 5   GAJ          918 non-null    int64  
 6   ECG          918 non-null    object 
 7   FCMAX        918 non-null    int64  
 8   ANGINE       918 non-null    object 
 9   DEPRESSION   918 non-null    float64
 10  PENTE        918 non-null    object 
 11  CŒUR         918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [6]:
# Affiche les prémières lignes de df.
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,homme,AA,140,289,0,Normal,172,Non,0.0,Ascendant,0
1,49,femme,DNA,160,180,0,Normal,156,Non,1.0,Plat,1
2,37,homme,AA,130,283,0,ST,98,Non,0.0,Ascendant,0
3,48,femme,ASY,138,214,0,Normal,108,Oui,1.5,Plat,1
4,54,homme,DNA,150,195,0,Normal,122,Non,0.0,Ascendant,0


#### Vérifie les doublons des lignes et les supprime.

In [7]:
# Compte le nombre de lignes dupliquées (ici il n'y en a pas).
df.duplicated().sum()

# Supprime les doublons.
# df.drop_duplicates()

0

#### Vérifie les données manquantes et les supprime.

In [8]:
# Compte le nombre de données manquantes par colonne (ici il n'y en a pas).
df.isna().sum()

# Supprime les données manquantes.
# df = df.drop_na(axis=0)

AGE            0
SEXE           0
TDT            0
PAR            0
CHOLESTEROL    0
GAJ            0
ECG            0
FCMAX          0
ANGINE         0
DEPRESSION     0
PENTE          0
CŒUR           0
dtype: int64

#### Vérifie les constantes et les supprime.

In [9]:
# Ici il n'y a pas de constantes, car chaque variable prend au moins deux valeurs différentes.
df.nunique()

# Supprime une constante.
# df = df.drop("nom_column", axis=1)

AGE             50
SEXE             2
TDT              4
PAR             67
CHOLESTEROL    222
GAJ              2
ECG              3
FCMAX          119
ANGINE           2
DEPRESSION      53
PENTE            3
CŒUR             2
dtype: int64

#### Normalise les variables quantitatives.

In [10]:
# Selectionne les colonnes de type number.
for column in df.drop("CŒUR", axis=1).select_dtypes(np.number).columns:
    df[column] = df[column] / df[column].max()

In [11]:
# Vérifie le résultat.
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,homme,AA,0.70,0.479270,0.0,Normal,0.851485,Non,0.000000,Ascendant,0
1,0.636364,femme,DNA,0.80,0.298507,0.0,Normal,0.772277,Non,0.161290,Plat,1
2,0.480519,homme,AA,0.65,0.469320,0.0,ST,0.485149,Non,0.000000,Ascendant,0
3,0.623377,femme,ASY,0.69,0.354892,0.0,Normal,0.534653,Oui,0.241935,Plat,1
4,0.701299,homme,DNA,0.75,0.323383,0.0,Normal,0.603960,Non,0.000000,Ascendant,0


#### Recode les variables qualitatives.

In [12]:
# Encode les variables qualitatives.
for column in df.drop("CŒUR", axis=1).select_dtypes("object").columns:
        df[column] = df[column].astype("category").cat.codes

In [13]:
# Vérifie le résultat.
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,1,0,0.70,0.479270,0.0,1,0.851485,0,0.000000,0,0
1,0.636364,0,3,0.80,0.298507,0.0,1,0.772277,0,0.161290,2,1
2,0.480519,1,0,0.65,0.469320,0.0,2,0.485149,0,0.000000,0,0
3,0.623377,0,1,0.69,0.354892,0.0,1,0.534653,1,0.241935,2,1
4,0.701299,1,3,0.75,0.323383,0.0,1,0.603960,0,0.000000,0,0


#### Crée deux objets : x contenant les variables AGE jusqu'à PENTE et y contenant uniquement CŒUR.

In [14]:
# Sépare df: x les variables explicatives et y la variable cible (coeur).
x = df.drop("CŒUR", axis=1)
y = df.CŒUR

In [15]:
# Vérifie le résultat.
x.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE
0,0.519481,1,0,0.70,0.479270,0.0,1,0.851485,0,0.000000,0
1,0.636364,0,3,0.80,0.298507,0.0,1,0.772277,0,0.161290,2
2,0.480519,1,0,0.65,0.469320,0.0,2,0.485149,0,0.000000,0
3,0.623377,0,1,0.69,0.354892,0.0,1,0.534653,1,0.241935,2
4,0.701299,1,3,0.75,0.323383,0.0,1,0.603960,0,0.000000,0


In [16]:
# Vérifie le résultat.
y.head()

0    0
1    1
2    0
3    1
4    0
Name: CŒUR, dtype: int64

#### Répartis les données du dataset.

In [17]:
# Répartis le jeu de données, 70% pour l'entraînement et 30% pour les tests.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

# train_set, test_set = train_test_split(df, test_size=0.3, random_state=0)

In [18]:
# Vérifie le résultat.
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(642, 11) (276, 11) (642,) (276,)


#### Entraîne le modèle de régression logistique.

In [19]:
# Instancie la classe de régression logistique.
logistic_regression = LogisticRegression(random_state=0)

In [20]:
# Entraîne (apprentissage) notre modèle de régression logistique.
model = logistic_regression.fit(x_train, y_train)

# model = logistic_regression.fit(train_set.drop("CŒUR", axis=1), train_set["CŒUR"])

In [21]:
# La probabilité d'appartenance à l'une des classes (0 - 1 , non - oui , pas malade - malade).
prediction_proba = model.predict_proba(x_test)
prediction_proba[:5, :]

# [non, oui]

array([[0.08258089, 0.91741911],
       [0.28850182, 0.71149818],
       [0.06013223, 0.93986777],
       [0.06178491, 0.93821509],
       [0.84059458, 0.15940542]])

In [22]:
# Applique le modèle au données de test.
y_pred = model.predict(x_test)
y_pred[:5]

array([1, 1, 1, 1, 0], dtype=int64)

#### Conçois la matrice de confusion, la précision et la sensibilité.

#### La matrce de confusion

In [23]:
"""
Selon la matrice de confusion, notre modèle prédit:
- 91 fois correctement qu'un individu a une maladie du coeur (vp).
- 22 fois de façon erronée qu'un individu a une maladie du coeur (fp).
- 26 fois de façon erronée qu'un individu n'a pas une maladie du coeur (fn).
- 137 fois correctement qu'un individu n'a pas une maladie du coeur (vn).

array([ [vp, fp]
        [fn, vn]])
"""

# La matrice de confusion.
confusion_matrice = confusion_matrix(y_test, y_pred)
confusion_matrice

array([[ 91,  22],
       [ 26, 137]], dtype=int64)

#### Le taux de bonnes prédictions

In [24]:
# Le taux de bonnes prédictions.
accuracy = accuracy_score(y_test, y_pred)
accuracy

# Interprétation: le modèle produit plus 82% de bonnes prédictions.

0.8260869565217391

#### La sensibilité

In [25]:
# La sensibilité.
recall = recall_score(y_test, y_pred)
recall

# vp / (vp + fn)
# recall = 91 / (91 + 26)

# Interprétation: lorsqu'un individu est malade, le modèle a plus de 84% de chance de le prédire.

0.8404907975460123

#### La précision

In [26]:
# La précision.
precision = precision_score(y_test, y_pred)
precision

# vp / (vp + fp)
# precision = 91 / (91 + 22)

# Interprétation: lorsque le modèle prédit qu'un individu est malade, il y a plus de 86% de chance que cela soit vraie.

0.8616352201257862

In [27]:
# Affiche plus de détails avec classification report.
report_classification = classification_report(y_test, y_pred)
print(report_classification)

              precision    recall  f1-score   support

           0       0.78      0.81      0.79       113
           1       0.86      0.84      0.85       163

    accuracy                           0.83       276
   macro avg       0.82      0.82      0.82       276
weighted avg       0.83      0.83      0.83       276



In [28]:
# Modèle Trival.
df.CŒUR.value_counts() / df.shape[0]

# Interprétation: si nous nous basons sur le hasard pour faire nos prédictions, nous aurons plus de 55% de bonnes prédictions.

1    0.553377
0    0.446623
Name: CŒUR, dtype: float64

In [29]:
# Le score d'entraînement: taux de bonnes prédictions calculé sur la base des données d'entraînement.
model.score(x_train, y_train)

0.8660436137071651

In [30]:
# Le score d'entraînement: le taux de bonnes prédictions calculé sur la base des données de test.
model.score(x_test, y_test)

0.8260869565217391

#### Entraîne le modèle d'arbre de décision.

In [33]:
# Instancie la classe d'arbre de décision.
decision_tree = DecisionTreeClassifier(random_state=0)

In [43]:
# Entraîne notre modèle d'arbre de décision.
model_decision_tree = decision_tree.fit(x_train, y_train)

In [44]:
# La probabilité d'appartenance à l'une des classes (0 - 1 , non - oui , pas malade - malade).
prediction_proba_decision_tree = model_decision_tree.predict_proba(x_test)
prediction_proba_decision_tree[:5, :]

# [non, oui]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [45]:
# Applique le modèle au données de test.
y_pred_decision_tree = model_decision_tree.predict(x_test)
y_pred_decision_tree[:5]

array([1, 1, 1, 1, 0], dtype=int64)

#### Conçois la matrice de confusion, la précision et la sensibilité.

#### La matrce de confusion

In [38]:
"""
Selon la matrice de confusion, notre modèle prédit:
- 90 fois correctement qu'un individu a une maladie du coeur (vp).
- 23 fois de façon erronée qu'un individu a une maladie du coeur (fp).
- 34 fois de façon erronée qu'un individu n'a pas une maladie du coeur (fn).
- 129 fois correctement qu'un individu n'a pas une maladie du coeur (vn).

array([ [vp, fp]
        [fn, vn]])
"""

# La matrice de confusion.
confusion_matrice_decision_tree = confusion_matrix(y_test, y_pred_decision_tree)
confusion_matrice_decision_tree

array([[ 90,  23],
       [ 34, 129]], dtype=int64)

#### Le taux de bonnes prédictions

In [39]:
# Le taux de bonnes prédictions.
accuracy_decision_tree = accuracy_score(y_test, y_pred_decision_tree)
accuracy_decision_tree

# Interprétation: le modèle produit plus 79% de bonnes prédictions.

0.7934782608695652

#### La sensibilité

In [40]:
# La sensibilité.
recall_decision_tree = recall_score(y_test, y_pred_decision_tree)
recall_decision_tree

# Interprétation: lorsqu'un individu est malade, le modèle a plus de 79% de chance de le prédire.

0.7914110429447853

#### La précision

In [41]:
# La précision.
precision_decision_tree = precision_score(y_test, y_pred_decision_tree)
precision_decision_tree

# Interprétation: lorsque le modèle prédit qu'un individu est malade, il y a plus de 84% de chance que cela soit vraie.

0.8486842105263158

In [42]:
# Affiche plus de détails avec classification report.
report_classification_decision_tree = classification_report(y_test, y_pred_decision_tree)
print(report_classification_decision_tree)

              precision    recall  f1-score   support

           0       0.73      0.80      0.76       113
           1       0.85      0.79      0.82       163

    accuracy                           0.79       276
   macro avg       0.79      0.79      0.79       276
weighted avg       0.80      0.79      0.79       276



In [46]:
# Le score d'entraînement: taux de bonnes prédictions calculé sur la base des données d'entraînement.
model_decision_tree.score(x_train, y_train)

1.0

In [47]:
# Le score d'entraînement: le taux de bonnes prédictions calculé sur la base des données de test.
model_decision_tree.score(x_test, y_test)

0.7934782608695652